# RL homework 5b

In [1]:
import tensorflow as tf
tf.__version__

'1.11.0'

## 1. Loss Function

### 1.1 Policy Loss

Case 1: **Reinforcement Learning Loss** 
$$ \nabla_{\phi} J_{\pi}(\phi) = \mathop{{}\mathbb{E}}_{s \sim D} \lbrace \mathop{{}\mathbb{E}}_{a \sim \pi_{\phi}(a|S)} [\nabla_{\phi} \log \pi(a|s) (\alpha \cdot log \pi_{\phi} (a|s) - Q_{\theta}(s, a)) + b(s) | s]  \rbrace $$ 

Case 2: **Reparameter Trick**
$$ \nabla_{\phi} J_{\pi}(\phi) = \mathop{{}\mathbb{E}}_{s \sim D} \lbrace \mathop{{}\mathbb{E}}_{\epsilon \sim N(0, I)} [\alpha \cdot \log \pi_{\phi}(f_{\phi}(\epsilon;s)|s) - Q_{\theta}(s, f_{\phi}(\epsilon; s)) | s] \rbrace $$ 

In [2]:
tf.stop_gradient

<function tensorflow.python.ops.gen_array_ops.stop_gradient(input, name=None)>

In [3]:
import tensorflow_probability as tfp
tfd = tfp.distributions
tfp.__version__

'0.4.0'

In [4]:
tfd.Normal

tensorflow.python.ops.distributions.normal.Normal

## 2. Squashing

### 2.1 Math Reasoning  
Reference: https://tex.stackexchange.com/questions/74125/how-do-i-put-text-over-symbols

* **Invertible functions' chain rule**

$$Z^{(N)} = (f_N \circ \cdots \circ f_1)(z^0) \iff \log p(z^{(N)}) = log(p(z^{(0)})) - \sum_{i=1}^{N} \left| \det(\frac{\partial f_i(z^{(i-1)})}{\partial z^{(i-1)}}) \right|$$

where $\frac{\partial f_i(z^{(i-1)})}{\partial z^{(i-1)}}$ is Jacobian of $f_i$, and $\det$ is the determinant.

* **Squashing via tanh for action A**

<!--
$$ a = \tanh \left(b_{\phi}(s) + A_{\phi}(s)\epsilon \right) \iff z=f_1(\epsilon) \triangleq b(s) + A(s) \epsilon, a = f_2(z) \triangleq \tanh(z) $$

or 

$$ a = \tanh \left(b_{\phi}(s) + A_{\phi}(s)\epsilon \right) \iff z=f_1(\epsilon) \equiv b(s) + A(s) \epsilon, a = f_2(z) \equiv \tanh(z) $$

or 
-->

$$a = \tanh \left(b_{\phi}(s) + A_{\phi}(s)\epsilon \right) \iff z=f_1(\epsilon) \stackrel{\text{def}}{=} b(s) + A(s) \epsilon, a = f_2(z) \stackrel{\text{def}}{=}\tanh(z) $$

As making f=$\tanh$, we have the Jacobian is a diagonal matrix with $\frac{\partial \tanh(z_i)}{\partial z_i} = 1 - \tanh ^2(z_i)$, finally we get

$$\log \left|  det(\frac{\partial f_2(z)}{\partial z}) \right| = \sum_{i=1}^{|A|} \log \left(1 - \tanh^2(z_i)\right)$$

Bonus Task:

$$\log \left(1 - \tanh^2(z_i)\right) = 2 \log 2 + 2 z_i - softplus(2z_i), \text{where } softplus(x) = \log(1+e^x)$$

## 3. SAC with Two Q-Functions

* Key points: using Q1 and Q2 with different parameter $\theta_1$ and $\theta_2$, then use $Q(s,a)=\min(Q_1(s,a), Q_2(s,a))$ to restrict the sampling upper bound.